# Error mitigation

This notebook contains theoretical background and examples for using our Error Mitigation module with Qiskit.

## Theoretical background
Here we describe main theoretical concepts related to mitigation procedure. For more detailed description, see Ref. [0].

### Classical noise model
Let us denote by $\mathbf{M}$ a POVM describing noisy detector and $\mathbf{P}$ denote ideal measurement. In classical noise model, we assume that the relation between the two is given by stochastic, invertible map $\Lambda$:

$$ \mathbf{M} = \Lambda \mathbf{P}.$$

Let us denote by $\mathbf{p}_{exp}$ vector of probabilities obtained on a noisy detector $\mathbf{M}$ measuring any quantum state, and by $\mathbf{p_{ideal}}$ the analogous vector for ideal detector (for the same quantum state). From linearity of the Born's rule, it follows that two vectors are related by the same stochastic map as POVMs:

$$ \mathbf{p_{exp}} = \Lambda \mathbf{p_{ideal}}. $$

Recall that we assumed that $\Lambda$ is invertible. Hence by multiplying last equation by $\Lambda^{-1}$ from both sides, we obtain

$$ \Lambda^{-1} \mathbf{p_{exp}} = \mathbf{p_{ideal}}. $$

Effectively, by this kind of postprocessing, we obtain statistics which we would have obtained on the perfect detector devices.

### Deviations from noise model
#### Effects  on mitigation
Above we assumed a very specific noise model. In practice it is likely that it will not be fulfilled exactly. In such a scenario, we may perform the following decomposition of POVM $M$ describing our device:
$$ \mathbf{M} = \Lambda \mathbf{P} + \mathbf{\Delta}\ ,$$
where $\Delta$ denotes a "coherent" part of the noise and $\Lambda$, as previously, is some stochastic, invertible map. 

In such a case, we can relate probability vector obtained on the noisy detector to ideal one in manner similar as before

$$ \mathbf{p_{exp}} = \Lambda \mathbf{p_{ideal}} + \mathbf{d}\ , $$
where $\mathbf{d}$ denotes a disturbance of probability vector due to existence of coherent part of the noise.
If we now multiply the above expression by inverse of the noise matrix, we obtain

$$ \Lambda^{-1} \mathbf{p_{exp}} = \mathbf{p_{ideal}} + \Lambda^{-1} \mathbf{d}, \  $$


TODO: finish

#### How to choose decomposition?
In general, we can always perform decomposition into "classical" and "coherent" part in infinitely many ways. However, for the ideal detector $P$ modeled as projective measurement in computational basis, there exists a perfectly natural ansatz. Namely, we propose to consider diagonal parts of POVM's $\mathbf{M}$ elements to describe classical part of the noise. 
As a justification for such choice, note that elements of $\mathbf{P}$ for such ideal detector model are diagonal, and the stochastic map would preserve the diagonality. 

Hence, after obtaining description of POVM $\mathbf{M}$ from detector tomography, reconstruction of $\Lambda$ can be achieved by taking only diagonal parts of POVM's elements.




### Finite-size statistics

### When is mitigation succesfull?










## Mitigating the error using our module

### Performing Quantum Detector Tomography
Our error mitigation approach is based on the knowledge about the noise in the device's detector. Such knowledge can be obtained in procedure known as Quantum Detector Tomography (QDT). To perform QDT, one can follow the steps from our [QDT tutorial](https://github.com/fbm2718/QREM/blob/master/QDT_Tutorial.ipynb). 


In [1]:
import povmtools
import ancillary_functions as anf
import numpy as np

from qiskit import IBMQ, Aer, execute
from qiskit.providers.aer import noise

from quantum_tomography_qiskit import detector_tomography_circuits
from DetectorTomographyFitter import DetectorTomographyFitter
from QDTErrorMitigator import QDTErrorMitigator

# Choose qubit indices
QDT_qubit_index = [3]

# Select probe kets
QDT_probe_kets = povmtools.pauli_probe_eigenkets

# Generate circuits
QDT_circuits = detector_tomography_circuits(QDT_qubit_index, QDT_probe_kets)

# Get QDT circuits results
backend = Aer.get_backend('qasm_simulator')  #  Get backed
shots_number = 2000  # Define number of measurement repetitions
QDT_job = execute(QDT_circuits, backend=backend, shots=shots_number)
results = QDT_job.result()

# Get ml_povm_estimator using DTF and results
DTF = DetectorTomographyFitter()
ml_povm_estimator = DTF.get_maximum_likelihood_povm_estimator([results], QDT_probe_kets)

## Preparing mitigation

Now that we have the estimator of POVM, we can create QDTErrorMitigator object and prepare it.

In [2]:
# Creation and preparation of QDTErrorMitigator
mitigator = QDTErrorMitigator()
mitigator.prepare_mitigator(ml_povm_estimator)

With prepared mitigator object we gain access to several useful functionalities. For example, we can:
* Correct results of qiskit job by using apply_correction_to_qiskit_job(Results) method.
* Access transition and correction matrices obtained from POVM given during preparation.

In order to properly analyse the results of correction procedure, one have to be aware that in some cases raw application of $\Lambda^{-1}$ to the results may yield quasiprobability (instead of probability) vectors. In such scenario our method calculates closest probability vectors and returns them instead. Distances from raw quasiprobabilities to returned probabilities, can be accessed via distances_from_closest_probability_vector member of mitigator object.

### Error bounds

With access to POVM and the correction and transition matrices, we are able to calculate bounds on several errors. In particular, using povtools module, we can calculate:
* statistical error bound (using get_statistical_error_bound method),
* coherent error bound (using gt_coherent_error_bound method),
* correction error bound (using get_correction_error_bound_from_data or get_correction_error_bound_from_parameters method).  

## Single qubit error mitigation scenario

In this section two examples of mitigation for single qubit will be shown. In order to show, that our mitigation scheme is efficient, we first need to create a noisy backend simulator. We start with required imports. 

In [3]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute
from qiskit import IBMQ, Aer
from qiskit.providers.aer.noise import NoiseModel

import povmtools
from DetectorTomographyFitter import DetectorTomographyFitter
from quantum_tomography_qiskit import detector_tomography_circuits
from QDTErrorMitigator import QDTErrorMitigator

We use standard qiskit methods to create noisy backend simulator. For this tutorial we will create simulator of IBM's Vigo device.

In [4]:
#  What I want to have first is working noisy backend simulation. In order to do that, I will use qiskit noise model
# and download properties of selected backend.

# Build noise model from backend properties.
provider = IBMQ.load_account()
backend = provider.get_backend('ibmq_vigo')
noise_model = NoiseModel.from_backend(backend)

# Get coupling map from backend, why not.
coupling_map = backend.configuration().coupling_map

# Get basis gates from noise model.
basis_gates = noise_model.basis_gates

# Finally, get the simulator backend.
simulator_backend = Aer.get_backend('qasm_simulator')

With backend created, what I need to do next is calculating POVM that describes its measurements best. To do that, following our QDT tutorial, I use DetectorTomographyFitter object from our module. In order to do that, I need to create calibration circuits and obtain their results first.

In [5]:
# What I want to do now, is prepare POVM for simulated backend. According to our other notebook, I prepare
# circuits first.

qdt_qubit_index = [0]
qdt_probe_kets = povmtools.pauli_probe_eigenkets
qdt_calibration_circuits = detector_tomography_circuits(qdt_qubit_index, qdt_probe_kets)

# I then execute them on backend prepared earlier.
shots_number = 2000

# Perform a noisy simulation
result = execute(qdt_calibration_circuits, simulator_backend, coupling_map=coupling_map, basis_gates=basis_gates,
                 noise_model=noise_model, shots=shots_number)\
                 .result()

# Print counts.
for i in range(len(result.results)):
    print(result.get_counts(i))

{'0': 1987, '1': 13}
{'0': 36, '1': 1964}
{'0': 1038, '1': 962}
{'0': 1043, '1': 957}
{'0': 994, '1': 1006}
{'0': 1006, '1': 994}


Shot's number is obviously a matter of choice in this case. It's preferable to use more (real backends allows more that 8k shots per job) as in that way we obtain better statistics. With more accurate job results, we can -- obviously -- prepare better POVMs. Maximum likelihood POVM calculation is as easy as calling single method now.

In [6]:
# With circuits results I can now use our Detector Tomography Fitter to obtain maximum likelihood POVM estimator.
dtf = DetectorTomographyFitter()
ml_povm_estimator = dtf.get_maximum_likelihood_povm_estimator([result], qdt_probe_kets)

for m_i in ml_povm_estimator:
    print(m_i)

[[ 0.99369959+7.67339058e-19j -0.00125087+2.99909517e-03j]
 [-0.00125087-2.99909517e-03j  0.01858137-9.63724971e-19j]]
[[0.00630041-7.67325823e-19j 0.00125087-2.99909517e-03j]
 [0.00125087+2.99909517e-03j 0.98141863-1.82198865e-16j]]


With POVM calculated, we can now create and prepare mitigator object.

In [7]:
# I can use obtained POVM to correct to prepare mitigation object.
mitigator = QDTErrorMitigator()
mitigator.prepare_mitigator(ml_povm_estimator)

With these preparations out of the way, we can now check how efficient our error mitigation approach is. Let's consider two simple scenarios.

### X gate circuit

In this case, we will first create a one qubit circuit (initially in |0><0| state) and we will apply X (not) operation to it. In ideal scenario we would expect all counts in state |1><1|. We begin with circuit creation.

In [8]:
# In order to check how efficient our mitigator is, we need to obtain some noisy data first.
# We will start with preparing simple experiment.

qr = QuantumRegister(1, 'qreg')
cr = ClassicalRegister(1, 'creg')
qc = QuantumCircuit(qr, cr)

qc.x(qr[0])
qc.measure(qr, cr)

Now we execute this circuit on our simulator and check the results.

In [9]:
result = execute(qc, simulator_backend, coupling_map=coupling_map, basis_gates=basis_gates,
                 noise_model=noise_model, shots=shots_number)\
                 .result()

for i in range(len(result.results)):
    print(result.get_counts(i))

{'0': 33, '1': 1967}


As we can see, noisy simulator returned some errors (|0><0| counts), as expected. Let's try to correct these results using our mitigator.

In [10]:
# Now let's correct them.
corrected_results = mitigator.apply_correction_to_qiskit_job(result)
print(corrected_results)

good format
[array([[0.],
       [1.]])]


What we see here, is that instead of counts, our mitigator returned frequencies. In order to obtain counts we could multiply  corrected results times number of shots. This step isn't obviously necessary, as it can be clearly seen that results are indeed better, but let's do it for good measure.

In [11]:
print([corrected_results[0][0][0] * shots_number, corrected_results[0][1][0] * shots_number])

[0.0, 2000.0]


Indeed, we've got barely one count, out of 2000, in calculated in wrong state. Compared to raw 52 counts, this is a very good outcome. Let's try another test.

### H gate circuit

In this case, we will use H (Hadamard) gate, instead of X. Everything else will be exactly like in the first scenario. We begin with creating and executing a circuit and then printing raw results of the job. We expect to obtain equal number of |1> and |0> states conuts.

In [17]:
# Let's try another example.
qr2 = QuantumRegister(1, 'qreg2')
cr2 = ClassicalRegister(1, 'creg2')
qc2 = QuantumCircuit(qr2, cr2)

qc2.h(qr2[0])
qc2.measure(qr2, cr2)

result = execute(qc2, simulator_backend, coupling_map=coupling_map, basis_gates=basis_gates,
                 noise_model=noise_model, shots=shots_number)\
                 .result()

for i in range(len(result.results)):
    print(result.get_counts(i))

{'0': 1030, '1': 970}


We see, that we again have got around 60 of wrong counts. Let's try to apply error mitigation.

In [18]:
# Now let's correct them.
corrected_results = mitigator.apply_correction_to_qiskit_job(result)
print(corrected_results)

good format
[array([[0.50908558],
       [0.49091442]])]


And print them as counts.

In [19]:
print([corrected_results[0][0][0] * shots_number, corrected_results[0][1][0] * shots_number])

[1018.1711624262215, 981.8288375737791]


Once again, the results are better!

There is, however one important thing concerning __Hadamard circuit__ in particular. Due to probabilistic nature of noisy simulation it's possible to obtain perfect (or close to perfect) outcome. In such scenario it's possible for out mitigation procedure to worsen the results. This effect can be observed even in this tutorial, for discussed outcomes of the circuit.

One can easily see, that determining if mitigation was a success or not is an important issue. This is especially true for experiments for which results are not known. Following procedure described in [0], by comparing operational distances and some error bounds, we can find out if mitigation failed or not! Let's do that. We start with calculating statistical error bound. We start with appointing mistake probability. 

In [20]:
# We can check out if mitigation was success. In order to do that, we have to calculate error bounds.
# Let's assume (according to the notation form the main paper) that alpha=0.

statistical_error_mistake_probability = 0.01

Now we need counts, formatted as array.

In [21]:
# We need counts as array. The outcomes are 0 or 1, so we can create them in straightforward way.
counts_dict = result.get_counts()
counts = [counts_dict['0'], counts_dict['1']]
print(counts)

[1030, 970]


Now we can use povmtools method to calculate statustical error bound.

In [22]:
# We calculate statistical error bound. Let's call it epsilon.
epsilon = povmtools.get_statistical_error_bound(counts, statistical_error_mistake_probability)
print(epsilon)

0.036394770800720934


That's, however, not enough. We'll also require correction error. Luckily, we have all the arguments necessary for its calculation using prepared povmtools method.

In [23]:
# We now need the correction error bound. Let's call it delta.
delta = povmtools.get_correction_error_bound_from_data_and_statistical_error(ml_povm_estimator,
                                                                             mitigator.correction_matrix, epsilon)
print(delta)

0.04115515278501394


Last thing required for determining mitigation success is operational distance between perfect measurement and our ml_povm_estimator. We calculate it, once again, using povmtools module function.

In [24]:
# We also need operational distance, between perfect detector and our POVM. We calculate it.
perfect_measurement = [[[1, 0], [0, 0]], [[0, 0], [0, 1]]]  # For one qubit!
operational_distance = povmtools.operational_distance_POVMs(ml_povm_estimator, perfect_measurement)
print(operational_distance)

0.018998749666391156


And finally, we can check if our mitigation was a success!

In [25]:
# Now we can check if mitigation can be deemed as success!
print(f'Mitigation success: {delta < operational_distance + epsilon}')

Mitigation success: True


## Multi Qubit error mitigation scenario

Our method can easily be expanded for multiple qubits. It's done analogically to the single qubit experiments. We start with preparing noisy backend, again. We can use the same code as in one qubit scenario.

In [26]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute
from qiskit import IBMQ, Aer
from qiskit.providers.aer.noise import NoiseModel

import povmtools
from DetectorTomographyFitter import DetectorTomographyFitter
from quantum_tomography_qiskit import detector_tomography_circuits
from QDTErrorMitigator import QDTErrorMitigator

#  What I want to have first is working noisy backend simulation. In order to do that, I will use qiskit noise model
# and download properties of selected backend.

# Build noise model from backend properties.
provider = IBMQ.load_account()
backend = provider.get_backend('ibmq_vigo')
noise_model = NoiseModel.from_backend(backend)

# Get coupling map from backend, why not.
coupling_map = backend.configuration().coupling_map

# Get basis gates from noise model.
basis_gates = noise_model.basis_gates

# Finally, get the simulator backend.
simulator_backend = Aer.get_backend('qasm_simulator')

Now the POVM. This time we need higher dimensional one, as we are concerning multi qubit scenario. The code, however, doesn't change much. The only thing we need to do is modify qdt_qubit_indices list.

In [27]:
# What I want to do now, is prepare POVM for simulated backend. According to our other notebook, I prepare
# circuits first.

qdt_qubit_indices = [0, 1]
qdt_probe_kets = povmtools.pauli_probe_eigenkets
qdt_calibration_circuits = detector_tomography_circuits(qdt_qubit_indices, qdt_probe_kets)

# I then execute them on backend prepared earlier.
shots_number = 8000

# Perform a noisy simulation
result = execute(qdt_calibration_circuits, simulator_backend, coupling_map=coupling_map, basis_gates=basis_gates,
                 noise_model=noise_model, shots=shots_number) \
    .result()

# Print counts.
for i in range(len(result.results)):
    print(result.get_counts(i))

{'00': 7914, '10': 59, '01': 27}
{'00': 202, '10': 7766, '11': 32}
{'00': 3987, '10': 3980, '11': 15, '01': 18}
{'00': 4108, '10': 3863, '11': 16, '01': 13}
{'00': 4149, '10': 3819, '11': 17, '01': 15}
{'00': 4082, '10': 3881, '11': 16, '01': 21}
{'00': 146, '11': 29, '01': 7825}
{'00': 4, '10': 139, '11': 7645, '01': 212}
{'00': 71, '10': 71, '11': 3864, '01': 3994}
{'00': 72, '10': 60, '11': 3893, '01': 3975}
{'00': 75, '10': 80, '11': 3829, '01': 4016}
{'00': 73, '10': 84, '11': 3787, '01': 4056}
{'00': 4030, '10': 17, '11': 26, '01': 3927}
{'00': 93, '10': 3889, '11': 3925, '01': 93}
{'00': 2107, '10': 1883, '11': 2002, '01': 2008}
{'00': 2048, '10': 1952, '11': 2011, '01': 1989}
{'00': 2068, '10': 1986, '11': 1975, '01': 1971}
{'00': 2099, '10': 1957, '11': 1983, '01': 1961}
{'00': 3989, '10': 23, '11': 21, '01': 3967}
{'00': 114, '10': 3912, '11': 3869, '01': 105}
{'00': 2012, '10': 2095, '11': 1960, '01': 1933}
{'00': 2002, '10': 2027, '11': 1937, '01': 2034}
{'00': 2050, '10': 

Now, we can calculate the POVM estimator.

In [28]:
# With circuits results I can now use our Detector Tomography Fitter to obtain maximum likelihood POVM estimator.
dtf = DetectorTomographyFitter()
ml_povm_estimator = dtf.get_maximum_likelihood_povm_estimator([result], qdt_probe_kets)

for m_i in ml_povm_estimator:
    print(m_i)

[[ 9.89770000e-01+1.11003668e-18j -1.69699034e-03-2.93625323e-03j
   3.38523772e-03-2.85113506e-03j  7.83420321e-04+2.53047563e-04j]
 [-1.69699034e-03+2.93625323e-03j  2.55854038e-02-6.70426578e-19j
   2.20772564e-03-1.17065036e-03j -9.73767163e-04+6.96014981e-05j]
 [ 3.38523772e-03+2.85113506e-03j  2.20772564e-03+1.17065036e-03j
   1.79574829e-02+4.12002120e-18j  1.00843009e-04+1.18286321e-04j]
 [ 7.83420321e-04-2.53047563e-04j -9.73767163e-04-6.96014981e-05j
   1.00843009e-04-1.18286321e-04j  4.44728104e-04-2.47757137e-19j]]
[[ 6.33870861e-03-4.57397792e-19j  1.60680854e-03+2.80335303e-03j
  -3.98888920e-04-7.14964947e-05j -4.16270627e-03-7.68820078e-03j]
 [ 1.60680854e-03-2.80335303e-03j  9.70239950e-01-5.74377277e-17j
  -3.73296228e-03+2.84503718e-03j -6.84455313e-03-3.24579678e-03j]
 [-3.98888920e-04+7.14964947e-05j -3.73296228e-03-2.84503718e-03j
   4.95132491e-05-3.41989552e-20j  4.67176614e-04+4.23075756e-04j]
 [-4.16270627e-03+7.68820078e-03j -6.84455313e-03+3.24579678e-03j
  

And with POVM estimator, we can prepare mitigator object.

In [30]:
# I can use obtained POVM to correct to prepare mitigation object.
mitigator = QDTErrorMitigator()
mitigator.prepare_mitigator(ml_povm_estimator)

Again, let's check how the mitigation work on several circuits.

### Hadamards circuit

We begin with circuit applying Hadamard gates on both qubits. Let's prepare such circuit.

In [31]:
# In order to check how efficient our mitigator is, we need to obtain some noisy data first.
# We will start with preparing simple experiment.

qr = QuantumRegister(2, 'qreg')
cr = ClassicalRegister(2, 'creg')
qc = QuantumCircuit(qr, cr)

qc.h(qr[0])
qc.h(qr[1])
qc.measure(qr, cr)

We then execute this circuit and print the results.

In [32]:
result = execute(qc, simulator_backend, coupling_map=coupling_map, basis_gates=basis_gates,
                 noise_model=noise_model, shots=shots_number) \
    .result()

for i in range(len(result.results)):
    print(result.get_counts(i))

{'00': 2064, '10': 2039, '11': 1894, '01': 2003}


Now we can correct them.

In [37]:
# Now let's correct them.
corrected_results = mitigator.apply_correction_to_qiskit_job(result)
print(corrected_results[0] * shots_number)

good format
[[1995.21199029]
 [1993.54915208]
 [2053.08697084]
 [1958.15188678]]


One can clarly see, that the results are better. Let's try with some multi-qubit gates.

### CNot Circuit

This time we will use a CNot gate. First we will prepare a qubit in state |1> and then use it as a control to negate the other qubit. We will expect all counts to be in state |11>. The circuit can be codded as below.

In [39]:
# Let's try another example.
qr2 = QuantumRegister(2, 'qreg2')
cr2 = ClassicalRegister(2, 'creg2')
qc2 = QuantumCircuit(qr2, cr2)

qc2.x(qr2[0])
qc2.cx(qr2[0], qr2[1])
qc2.measure(qr2, cr2)

Executing this circuits yields following results.

In [40]:
result = execute(qc2, simulator_backend, coupling_map=coupling_map, basis_gates=basis_gates,
                 noise_model=noise_model, shots=shots_number) \
    .result()

for i in range(len(result.results)):
    print(result.get_counts(i))

{'00': 16, '10': 122, '11': 7618, '01': 244}


As expected, we can see some garbage. Let's try to mitigate it.

In [41]:
# Now let's correct them.
corrected_results = mitigator.apply_correction_to_qiskit_job(result)
print(corrected_results[0] * shots_number)

good format
[[6.21386776e+00]
 [4.29210985e+01]
 [0.00000000e+00]
 [7.95086503e+03]]


Easily enough, the results are clearly better. Let's now combine Hadamard and CNot gate.

### Hadamard + CNot circuit

In this scenario, we will first prepare first qubit in superposition of states |0> and |1>. We will then, again, use it as a control qubit to negate the second one. We will expect equal number of counts in |00> and |11> states. The circuit can be prepared like below.

In [42]:
# Final example!
qr3 = QuantumRegister(2, 'qreg2')
cr3 = ClassicalRegister(2, 'creg2')
qc3 = QuantumCircuit(qr3, cr3)

qc3.h(qr3[0])
qc3.cx(qr3[0], qr2[1])
qc3.measure(qr3, cr3)

We now execute it and print the results.

In [43]:
result = execute(qc3, simulator_backend, coupling_map=coupling_map, basis_gates=basis_gates,
                 noise_model=noise_model, shots=shots_number) \
    .result()

for i in range(len(result.results)):
    print(result.get_counts(i))

{'00': 3928, '10': 90, '11': 3856, '01': 126}


There's some room for improvement. Let's apply the mitigation.

In [44]:
# Now let's correct them.
corrected_results = mitigator.apply_correction_to_qiskit_job(result)
print(corrected_results[0] * shots_number)

good format
[[3964.79532806]
 [   9.74310772]
 [   0.        ]
 [4025.46156423]]


Once again, the results are clearly better.

This time it was possible, to determine if the mitigation worked without using mitigation success check. If, however, one would like to perform it, then the only difference between multi and single qubit scenario code, would be perfect_measurement definition (adequate to the dimension of the problem). Mitigation success check won't be, therefore, shown for multi qubit case.

As a final remark one should be aware, that the results of the mitigation and its success in particular, strongly depends on quantum detector tomography method used during POVM calculation. In our work maximum likelihood method was used, but there are several different

# References

[0] Filip B. Maciejewski, Zoltán Zimborás, Michał Oszmaniec, *Mitigation of readout noise in near-term quantum devices by classical post-processing based on detector tomography*, arxiv preprint, https://arxiv.org/abs/1907.08518 (2019)
  